<a href="https://colab.research.google.com/github/veren4/SMILES_featurization/blob/master/LSTM_KDNuggets_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook ist based on [this tutorial](https://www.kdnuggets.com/2020/07/pytorch-lstm-text-generation-tutorial.html). The code is from their Github repo.

Open problems with this model:
* Wenn ich in dem Satz, den ich hinten reinfüttere zum Predicten, ein Zeichen habe, das im Trainings-Datensatz nicht vorkam, kriege ich einen Fehler. => Generell muss ich unknown tokens einführen.
* Ich schaue das Vokabular des ganzen Datensatzes an. Wenn ich den aber am Anfang nicht einlese, geht das nicht => Vorher bestimmen und hier nur einlesen!
* Datensatz einlesen, ohne komplett in den Cache zu laden ([Massive Dataset class](https://github.com/pytorch/text/issues/130))
* Adapt lstm size and embedding dim?

#Setup

In [1]:
import torch
import pandas as pd
from collections import Counter
from torch import nn, optim
import argparse
import numpy as np
from torch.utils.data import DataLoader

!pip install -q SmilesPE
from SmilesPE.pretokenizer import atomwise_tokenizer
import pickle

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import platform
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print('Python: ', platform.python_version())
print('PyTorch: ', torch.__version__)
if(device.type == 'cuda'):
  print('Using GPU (cuda)')
else:
  print('Using CPU!')

Python:  3.6.9
PyTorch:  1.7.0+cu101
Using GPU (cuda)


#Metdata

* Number of lines
* regex: cut away line number + tabs/spaces
* byte offset 

wc -l CID-SMILES\
108826964 CID-SMILES

108 826 964   lines are in that file.

In [4]:
'''
data_stream = open('/content/drive/My Drive/Rostlab internship/8_KDNuggets_LSTM_Approach/data/CID-SMILES', 'r')
#data_stream = open('/content/drive/My Drive/Rostlab internship/8_KDNuggets_LSTM_Approach/data/worded_smiles_no_header.csv', 'r')

offset_list = list()

number_of_lines = 108826964
#number_of_lines = 14

for i in range(number_of_lines):
  data_stream.readline()
  offset_list.append(data_stream.tell())

data_stream.close()

with open('/content/drive/My Drive/Rostlab internship/8_KDNuggets_LSTM_Approach/data/offset_list.pkl', 'wb') as fid:
#with open('/content/drive/My Drive/Rostlab internship/8_KDNuggets_LSTM_Approach/data/offset_list_worded_smiles_no_header.pkl', 'wb') as fid:

  pickle.dump(offset_list, fid)
'''

In [5]:
#with open('/content/drive/My Drive/Rostlab internship/8_KDNuggets_LSTM_Approach/data/offset_list.pkl', 'rb') as fid:
#     offset_list = pickle.load(fid)

with open('/content/drive/My Drive/Rostlab internship/8_KDNuggets_LSTM_Approach/data/offset_list_worded_smiles_no_header.pkl', 'rb') as fid:
     offset_list = pickle.load(fid)

In [6]:
byte_offset = {'byte_offset': offset_list}
df_byte_offset = pd.DataFrame(byte_offset)

#Dataset

##Standard Dataset class

In [4]:
#import torch
#import pandas as pd


class Dataset(torch.utils.data.Dataset):
    def __init__(
        self,
        args,
    ):
        self.args = args
        self.words = self.load_words()
        self.uniq_words = ['UNK', 'SOL', 'EOL', 'PAD', '1', 'N', ')', 'C', 'S', '=', '4', 'O', '(', '2', '3', 'P']

        # tokenization dictionaries (numerization)
        self.index_to_word = {index: word for index, word in enumerate(self.uniq_words)}
        self.word_to_index = {word: index for index, word in enumerate(self.uniq_words)}

        # numericalize all the tokens
        #self.words_indexes = [self.word_to_index[w] for w in self.words]
        # if the token is in word_to index, then the index, otherwise 'UNK' = 0
        # [f(x) if condition else g(x) for x in sequence]
        #self.words_indexes = [self.word_to_index[w] if (w in uniq_words) else self.word_to_index['UNK'] for w in self.words]
        self.words_indexes = [self.word_to_index[w] if (w in self.uniq_words) else 0 for w in self.words]

    def load_words(self):

        #train_df = pd.read_csv('data/worded_smiles.csv')
        infile = '/content/drive/My Drive/Rostlab internship/8_KDNuggets_LSTM_Approach/data/worded_smiles.csv'
        with open(infile, "r") as file1:
            train_df = pd.read_csv(file1)
        file1.close()

        # Tokenize
        train_df['tokenized_SMILES'] = ''
        for row in range(train_df.shape[0]):
          train_df.loc[row, 'tokenized_SMILES'] = atomwise_tokenizer(train_df.loc[row, 'SMILES'])
        
        # Padding + SOL + EOL
        for row in range(train_df.shape[0]):              # ATTENTION: Are the column indexes correct?!
          actual_length = len(train_df.loc[row, 'tokenized_SMILES'])
          length_before_delimiters = self.args.sequence_length - 2

          if actual_length > length_before_delimiters:
            train_df.loc[row, 'tokenized_SMILES'] = train_df.loc[row, 'tokenized_SMILES'][:length_before_delimiters]
            train_df.loc[row, 'tokenized_SMILES'].append('EOL')
          elif actual_length < length_before_delimiters:
            temp = ['UNK']*length_before_delimiters
            shortie = train_df.loc[row, 'tokenized_SMILES']
            shortie.append('EOL')
            temp[:actual_length] = shortie
            train_df.loc[row, 'tokenized_SMILES'] = temp
          train_df.loc[row, 'tokenized_SMILES'].insert(0, 'SOL')
          
        # return the whole dataset as 1 list of tokens
        total_token_list = []
        for row in range(train_df.shape[0]):
          total_token_list.extend(train_df.loc[row, 'tokenized_SMILES'])      # can be combined with above.
        return total_token_list

    def __len__(self):
        return len(self.words_indexes) - self.args.sequence_length   # !!!!

    def __getitem__(self, index):

        return (
            torch.tensor(self.words_indexes[index:index+self.args.sequence_length]),  # input      # turn into cuda tensor?
            torch.tensor(self.words_indexes[index+1:index+self.args.sequence_length+1]),    # label
        )

##Testing

0 = 'UNK'\
1 = 'SOL'\
2 = 'EOL'\
3 = 'PAD'

Es sollte als jeder Batch mit 1 beginnen, und mit 2 oder 3 enden.

In [ ]:
dataset = Dataset(args)
subset_indices = [0]

subset = torch.utils.data.Subset(dataset, subset_indices)

#testloader_subset = torch.utils.data.DataLoader(subset, batch_size=args.batch_size, num_workers=0, shuffle=False)
testloader_subset = DataLoader(subset, batch_size=args.batch_size, shuffle=False)

for batch, (x, y) in enumerate(testloader_subset):
            print('\nbatch: ', batch, '\n(x, y):', (x, y))

In [ ]:
#dataset.args.batch_size

In [ ]:
#for batch, (x, y) in enumerate(dataloader):
#            print('\nbatch: ', batch, '\n(x, y):', (x, y))

##Massive

In [ ]:
class Massive_Dataset(torch.utils.data.Dataset):
    def __init__(
        self,
        args,
    ):
        self.args = args
        #self.words = self.load_words()
        self.uniq_words = ['UNK', 'SOL', 'EOL', '1', 'N', ')', 'C', 'S', '=', '4', 'O', '(', '2', '3', 'P']
        self.offsets = offset_meta
        self.length_of_dataset = dataset_length

        # tokenization dictionaries (numerization)
        self.index_to_word = {index: word for index, word in enumerate(self.uniq_words)}
        self.word_to_index = {word: index for index, word in enumerate(self.uniq_words)}

        # numericalize all the tokens
        # if the token is in word_to index, then the index, otherwise 'UNK' = 0
        self.words_indexes = [self.word_to_index[w] if (w in self.uniq_words) else self.word_to_index['UNK'] for w in self.words]

        self.data_path = '/content/drive/My Drive/Rostlab internship/8_KDNuggets_LSTM_Approach/data/worded_smiles_no_header.csv'
        # should be reset in __iter__
        self.data_stream = open(self.data_path, 'r')
        self.current_offset = 0

        # provide Metadata: byte offsets, # of instances


    def __line_to_instance__(self):    #load_words(self):

        # Tokenize
        tokenized_SMILES = atomwise_tokenizer(line)
        
        # Padding + SOL + EOL
        actual_length = len(tokenized_SMILES)
        length_before_delimiters = self.args.sequence_length - 2

        tokenized_SMILES.insert(0, 'SOL')
        tokenized_SMILES.append('EOL')
        
        return tokenized_SMILES

    def __len__(self):
        return len(self.words_indexes) - self.args.sequence_length  # Why is the sequence length substracted?



    def __getitem__(self, index):     # Should I do something with the index?

        # attention: not len, but bytes!
        #offset = ??

        # Create metadata: offset, # samples
        
        self.data_stream.seek(offset)
        line = self.data_stream.readline()
        instance = self.line_to_instance(line)
        offset = offset + len(line)

        # reset to previous location for iteration
        self.data_stream.seek(self.current_offset)      # (Why) Do I need this? When testing, I could jump back without problems.
        
        return (           
            torch.tensor(self.words_indexes[instance:instance+self.args.sequence_length]),        # turn into cuda tensor?
            torch.tensor(self.words_indexes[instance+1:instance+self.args.sequence_length+1]),
        )
        
    # Sould I close self.data_stream after training further down?

In [ ]:
dataset = Massive_Dataset(args,
                          offset_meta = df_byte_offset,
                          #dataset_length = 108826964
                          dataset_length = 14)

#Model

In [5]:
#import torch
#from torch import nn

class Model(nn.Module):
    def __init__(self, dataset):
        super(Model, self).__init__()
        self.lstm_size = 128
        self.embedding_dim = 128
        self.num_layers = 3 #1

        n_vocab = len(dataset.uniq_words)       # provide predermined number here!
        self.embedding = nn.Embedding(
            num_embeddings=n_vocab,
            embedding_dim=self.embedding_dim,
        )
        self.lstm = nn.LSTM(
            input_size=self.lstm_size,
            hidden_size=self.lstm_size,
            num_layers=self.num_layers,
            dropout=0.2#,  #0,
            #bidirectional=True
        )
        self.fc = nn.Linear(self.lstm_size, n_vocab)

    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        logits = self.fc(output)

        return logits, state

    def init_state(self, sequence_length):
        return (torch.zeros(self.num_layers, sequence_length, self.lstm_size),
                torch.zeros(self.num_layers, sequence_length, self.lstm_size))
                

    #def get_hidden_state(self, x):
    #    out, (h_n, c_n) = self.lstm(embed, prev_state)      # problem: usually, I provide the entire dataset as an input here. That is not possible. => I need to extract this information at the end of the training.
    #    return h_n, c_n
        

#Train

In [6]:
#import argparse
#import torch
#import numpy as np
#from torch import nn, optim
#from torch.utils.data import DataLoader

def train(dataset, model, args):
    model.train()       # set the model to training mode

    dataloader = DataLoader(
      dataset,
      batch_size=args.batch_size,
    )

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(args.max_epochs):
        state_h, state_c = model.init_state(args.sequence_length)

        for batch, (x, y) in enumerate(dataloader):

            optimizer.zero_grad()

            y_pred, (state_h, state_c) = model(x, (state_h, state_c))
            loss = criterion(y_pred.transpose(1, 2), y)

            state_h = state_h.detach()
            state_c = state_c.detach()

            loss.backward()
            optimizer.step()

            print({ 'epoch': epoch, 'batch': batch, 'loss': loss.item() })
    

def predict(dataset, model, text, next_words=100):   # use this function to output the states at all timesteps!
    words = text.split(' ')
    model.eval()

    state_h, state_c = model.init_state(len(words))

    for i in range(0, next_words):
        x = torch.tensor([[dataset.word_to_index[w] for w in words[i:]]])
        y_pred, (state_h, state_c) = model(x, (state_h, state_c))

        last_word_logits = y_pred[0][-1]
        p = torch.nn.functional.softmax(last_word_logits, dim=0).detach().numpy()
        word_index = np.random.choice(len(last_word_logits), p=p)
        words.append(dataset.index_to_word[word_index])
        
    return words

#Apply

In [10]:
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--max-epochs', type=int, default=10)
parser.add_argument('--batch-size', type=int, default=3)
parser.add_argument('--sequence-length', type=int, default=150)
args = parser.parse_args("")

Change the dataset in dataset.py.\
Set max epochs, batch size, seqence length in train.py.

In [10]:
dataset = Dataset(args)
model = Model(dataset)  # to cuda

In [ ]:
#dataset = Massive_Dataset(args)
#model = Model(dataset)

In [11]:
train(dataset, model, args)

Streaming output truncated to the last 5000 lines.
{'epoch': 2, 'batch': 200, 'loss': 0.05635999143123627}
{'epoch': 2, 'batch': 201, 'loss': 0.10902570188045502}
{'epoch': 2, 'batch': 202, 'loss': 0.12709645926952362}
{'epoch': 2, 'batch': 203, 'loss': 0.15640504658222198}
{'epoch': 2, 'batch': 204, 'loss': 0.1836673468351364}
{'epoch': 2, 'batch': 205, 'loss': 0.19546209275722504}
{'epoch': 2, 'batch': 206, 'loss': 0.25132203102111816}
{'epoch': 2, 'batch': 207, 'loss': 0.25348997116088867}
{'epoch': 2, 'batch': 208, 'loss': 0.2213210165500641}
{'epoch': 2, 'batch': 209, 'loss': 0.18987905979156494}
{'epoch': 2, 'batch': 210, 'loss': 0.1663333922624588}
{'epoch': 2, 'batch': 211, 'loss': 0.14255060255527496}
{'epoch': 2, 'batch': 212, 'loss': 0.1134936586022377}
{'epoch': 2, 'batch': 213, 'loss': 0.09565788507461548}
{'epoch': 2, 'batch': 214, 'loss': 0.09103257209062576}
{'epoch': 2, 'batch': 215, 'loss': 0.08372525870800018}
{'epoch': 2, 'batch': 216, 'loss': 0.07084140926599503}
{

In [12]:
print(predict(dataset, model, text='C C ( O 2 ) C'))

['C', 'C', '(', 'O', '2', ')', 'C', '(', '=', 'O', ')', 'C', 'C', 'N', 'C', '(', '=', 'O', ')', 'C', '(', 'C', '(', 'C', ')', '(', 'C', ')', 'C', 'O', 'P', '(', '=', 'O', ')', '(', 'O', ')', 'O', ')', 'C', 'EOL', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'SOL', 'C', 'C', '(', '=', 'C', 'C', 'C', 'C', '(', '=', 'C', 'C', 'C', 'C', '(', '=', 'C', 'C', 'C', 'C', '(', '=', 'C', 'C', 'C', '1', '=', 'C', '(', 'C', '=', 'C', 'C', '(']


In [ ]:
#last_state_h

In [ ]:
#last_state_c

# Visualization

In [ ]:
!nvidia-smi 